### MICS IRAQ

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/LFS')
[f for f in os.listdir() if 'sav' in f]

['LFS_2021.sav']

In [3]:
#read in data and meta
df_lfs,meta_lfs=pyreadstat.read_sav('LFS_2021.sav', apply_value_formats=False)

col_names_lfs=meta_lfs.column_names_to_labels
col_vals_lfs=meta_lfs.variable_value_labels

In [63]:
df=df_lfs.copy()

#having_disability
'''
gen SSHD=0
replace SSHD = . if VIS_SS==. & HEAR_SS ==. & MOB_SS ==. & COM_SS==. & UB_SS==. & COG_SS==.
replace SSHD = 2 if VIS_SS==2 | HEAR_SS ==2 | MOB_SS ==2 | COM_SS==2 |UB_SS==2 | COG_SS==2
replace SSHD = 3 if VIS_SS==3 | HEAR_SS ==3 | MOB_SS ==3 | COM_SS==3 |UB_SS==3 | COG_SS==3
replace SSHD = 4 if VIS_SS==4 | HEAR_SS ==4 | MOB_SS ==4 | COM_SS==4 |UB_SS==4 | COG_SS==4
replace SSHD = 1 if SSHD ==0
drop if SSHD ==.
label define sshdlb 1 "None" 2 "Mild" 3 "Moderate" 4 "Severe"
label values SSHD sshdlb
'''

having_disability_labels={1.0: 'No, no difficulty',
 2.0: 'Yes, some difficulty',
 3.0: 'Yes, a lot of difficulty',
 4.0: 'Cannot do it at all',
 np.nan:'Not stated'}

dis_cols=['dif_sight','dif_hear','dif_mobi','dif_conc','dif_comm','dif_upper_body','dif_care']
df['having_disability']=df[dis_cols].apply(lambda x: x.max(), axis=1)
df['having_disability']=df['having_disability'].map(having_disability_labels)
#####################################################
#number of functional disability
''' 
{1.0: 'No, no difficulty',
 2.0: 'Yes, some difficulty',
 3.0: 'Yes, a lot of difficulty',
 4.0: 'Cannot do it at all'}
 
 if any of the disabilities have codes 1 or 2 they are not disabled, disabled otherwise
 -transform codes 3,4 to True and 1,2 to False and sum over axis=1'''

df['func_dis']=df[dis_cols].apply(lambda x: x.isin([3,4]), axis=1).sum(axis=1)
num_functional_dis_labels={0:'Zero domain', 1:'One domain', 2:'Two domains', 3:'Three or more domains'}
df['num_functional_dis']=np.select([df['func_dis']==1, df['func_dis']==2, df['func_dis']>=3], [1,2,3])
df['num_functional_dis']=df['num_functional_dis'].map(num_functional_dis_labels)
print('number of functional disability created')
#####################################################
#household type
'''
{1.0: 'Head',
 2.0: 'Spouse (Wife/Husband)',
 3.0: 'Child (Son/daughter)',
 4.0: 'Grandchild',
 5.0: 'Parent',
 6.0: 'Brother or Sister',
 7.0: 'Son or Daughter in law',
 8.0: 'Other relative',
 9.0: 'Non-relative',
 10.0: 'Domestic worker /Maid/Servant (Live-in)'}'''

hh_type_labels={1:'One person', 2:'Nuclear', 3:'Extended', 4:'Composite', 5: 'Unknown'}
def family_type(df):
    nuclear=[1,2,3]
    extended=nuclear+[4,5,6,7,8]
    composite=extended+[9,10]

    if len(df['dem_rel'])==1:
        df['hh_type']=1
    elif all(df['dem_rel'].isin(nuclear)):
        df['hh_type']=2
    elif all(df['dem_rel'].isin(extended)):
        df['hh_type']=3
    elif all(df['dem_rel'].isin(composite)):
        df['hh_type']=4
    else:
        df['hh_type']=5
    return(df)
        
#create hh_type
#grouping by 2 variables will create a MultiIndex df
df=df.groupby(['cls_num','household_number','area']).apply(family_type)
df['hh_type']=df['hh_type'].map(hh_type_labels)
print('hh_type created')
#####################################################
#agegroup1: 0-4 5-9 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49 50-54 55-59 60-64 65-69 70-74 75-7 80-84 85-89 90+

c1=(df['dem_ag1']>=0) & (df['dem_ag1']<=4)
c2=(df['dem_ag1']>=5) & (df['dem_ag1']<=9)
c3=(df['dem_ag1']>=10) & (df['dem_ag1']<=14)
c4=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c5=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c6=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c7=(df['dem_ag1']>=30) & (df['dem_ag1']<=34)
c8=(df['dem_ag1']>=35) & (df['dem_ag1']<=39)
c9=(df['dem_ag1']>=40) & (df['dem_ag1']<=44)
c10=(df['dem_ag1']>=45) & (df['dem_ag1']<=49)
c11=(df['dem_ag1']>=50) & (df['dem_ag1']<=54)
c12=(df['dem_ag1']>=55) & (df['dem_ag1']<=59)
c13=(df['dem_ag1']>=60) & (df['dem_ag1']<=64)
c14=(df['dem_ag1']>=65) & (df['dem_ag1']<=69)
c15=(df['dem_ag1']>=70) & (df['dem_ag1']<=74)
c16=(df['dem_ag1']>=75) & (df['dem_ag1']<=79)
c17=(df['dem_ag1']>=80) & (df['dem_ag1']<=84)
c18=(df['dem_ag1']>=85) & (df['dem_ag1']<=89)
c19=(df['dem_ag1']>=90)

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19]

rslt=['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
'40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84','85-89', '90+']

df['agegroup1']=np.select(cond,rslt,default=np.nan)
print('agegroup1 created')
#####################################################
#agegroup2: 18+
df['agegroup2']=np.select([df['dem_ag1']>=18, df['dem_ag1']<18],['18+','<18'],default=np.nan )
print('agegroup2 created')
#####################################################
#agegroup3: 5-14	15-24	25-34	35-44	45-54	55-64	65-74	75-84	85+
c1=(df['dem_ag1']>=5) & (df['dem_ag1']<=14)
c2=(df['dem_ag1']>=15) & (df['dem_ag1']<=24)
c3=(df['dem_ag1']>=25) & (df['dem_ag1']<=34)
c4=(df['dem_ag1']>=35) & (df['dem_ag1']<=44)
c5=(df['dem_ag1']>=45) & (df['dem_ag1']<=54)
c6=(df['dem_ag1']>=55) & (df['dem_ag1']<=64)
c7=(df['dem_ag1']>=65) & (df['dem_ag1']<=74)
c8=(df['dem_ag1']>=75) & (df['dem_ag1']<=84)
c9=df['dem_ag1']>=85

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9]
rslt=['5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']
df['agegroup3']=np.select(cond,rslt,default=np.nan)
print('agegroup3 created')
#####################################################
# agegroup4: <15	15-19	20-24	25-29	30-34	35-39	40-44	45-49	50-54	55-59	60-64	65+
c1=df['dem_ag1']<15
c2=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c3=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c4=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c5=(df['dem_ag1']>=30) & (df['dem_ag1']<=34)
c6=(df['dem_ag1']>=35) & (df['dem_ag1']<=39)
c7=(df['dem_ag1']>=40) & (df['dem_ag1']<=44)
c8=(df['dem_ag1']>=45) & (df['dem_ag1']<=49)
c9=(df['dem_ag1']>=50) & (df['dem_ag1']<=54)
c10=(df['dem_ag1']>=55) & (df['dem_ag1']<=59)
c11=(df['dem_ag1']>=60) & (df['dem_ag1']<=64)
c12=df['dem_ag1']>=65

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12]
rslt=['<15','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65+']
df['agegroup4']=np.select(cond,rslt,default=np.nan)
print('agegroup4 created')
#####################################################
#agegroup5 <18	18-24	25-64	65+
c1=df['dem_ag1']<18
c2=(df['dem_ag1']>=18) & (df['dem_ag1']<=24)
c3=(df['dem_ag1']>=25) & (df['dem_ag1']<=64)
c4=df['dem_ag1']>=65

cond=[c1,c2,c3,c4]
rslt=['<18','18-24','25-64','65+']
df['agegroup5']=np.select(cond,rslt,default=np.nan)
print('agegroup5 created')
#####################################################
#age5+, age15+, age25+
df['age5+']=np.where(df['dem_ag1']>=5, '5+', '<5')
df['age15+']=np.where(df['dem_ag1']>=15, '15+', '<15')
df['age25+']=np.where(df['dem_ag1']>=5, '25+', '<25')
#####################################################
#agegroup6 15-29	30-44	45-64	65+
c1=(df['dem_ag1']>=15) & (df['dem_ag1']<=29)
c2=(df['dem_ag1']>=30) & (df['dem_ag1']<=44)
c3=(df['dem_ag1']>=45) & (df['dem_ag1']<=64)
c4=df['dem_ag1']>=65

cond=[c1,c2,c3,c4]
rslt=['15-29','30-44','45-64','65+']
df['agegroup6']=np.select(cond,rslt,default=np.nan)
print('agegroup6 created')
#####################################################

#####################################################
#relationship to HH
c1=df['dem_rel']==1
c2=df['dem_rel'].isin([2,3,4,5,6,7,8,9,10])
df['hh_relationship']=np.select([c1,c2],[1,2],default=999)
df['hh_relationship']=df['hh_relationship'].map({1:'head of HH', 2:'Other', 999:'Not stated'})
print('hh_relationship created')
#####################################################
#household size: 1	2	3	4	5	6	7	8+
df['hh_size']=np.where(df['ghhhsize']>=8, '8+',df['ghhhsize'])
print('hh_size created')
#####################################################
#add labels
vars=['dem_sex','area','dif_sight','dif_hear','dif_mobi','dif_conc','dif_care','dif_comm','dif_upper_body','dif_sight_reason','dif_hear_reason','dif_mobi_reason','dif_comm_reason','dif_conc_reason','dif_care_reason','dif_upper_body_reason','dem_mrt','ilo_edu_isced11_Yves']

for v in vars:
    df[v]=df[v].map(col_vals_lfs[v])

number of functional disability created
hh_type created
agegroup1 created
agegroup2 created
agegroup3 created
agegroup4 created
agegroup5 created
agegroup6 created
hh_relationship created
hh_size created


In [90]:
#to supress scientific notation
pd.options.display.float_format = '{:.2f}'.format
######################################################################################################
newcol=['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49','50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85-89', '90+', '18+', '<18']

xtab_total=[]
for col in ['agegroup1', 'agegroup2']:

       xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df[col],
       rownames=['Area','Disability Type', 'Sex'],
       colnames=['agegroup'], 
       values=df['weight'], aggfunc='sum',dropna=False)
       #drop the column 'na' from the dataframe
       xtab.drop(['nan'], axis=1, inplace=True)
       xtab_total.append(xtab)

#append the dataframes
T=pd.concat(xtab_total, axis=1) 

#reindex the columns to rearrange them
T=T.reindex(newcol, axis='columns')
  
T.to_excel('Table 1 xtab_all_dis_ByAge.xlsx')
print('Table 1: population_age generated and saved')

######################################################################################################
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
newcol=['5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']

xtab_total=[]
for dis in dis_cols:
    #filter dataframe
    df1=df[df[dis].isin(['Yes, a lot of difficulty','Cannot do it at all'])].copy()
    xtab=pd.crosstab([df1['area'],df1[dis],df1['dem_sex']],df1['agegroup3'],
    rownames=['Area','Disability Type', 'Sex'],
    colnames=['agegroup'], 
    values=df['weight'], aggfunc='sum',dropna=False)

    xtab=xtab.reindex(newcol, axis='columns')
    #add the name of the disability as an index
    # T['disability'] = dis
    # T.set_index('disability', append=True, inplace=True)

    #add the diability name as a new level on the columns
    #replicate the disability name as much as the column length
    #zip to create tuples and then use them as the new columns
    head1=[dis]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)

T.to_excel('Table 2 DisabType_Age.xlsx')
print('Table 2: DisabType_Age generated and saved')

#############################################################################
dis_reasons=['dif_sight_reason','dif_hear_reason','dif_mobi_reason','dif_comm_reason','dif_conc_reason','dif_care_reason','dif_upper_body_reason']

xtab_total=[]
for r in dis_reasons:
    #map labels
    xtab=pd.crosstab([df['area'],df['dem_sex']],df[r],
    rownames=['Area','Sex'],
    colnames=[r],
    values=df['weight'], aggfunc='sum',dropna=False)
    #add column name as multiindex column
    head1=[r]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)
T.to_excel('Table 3 CauseType.xlsx')
print('Table 3: CauseType generated and saved')

#############################################################################
xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df['num_functional_dis'],
rownames=['Area','Disability Type', 'Sex'],
colnames=['num_functional_dis'], 
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 4 Number_DisabilityDomain.xlsx')
print('Table 4: Number_DisabilityDomain generated and saved')
#############################################################################
xtab=pd.crosstab([df['area'],df['dem_mrt'],df['having_disability'],df['dem_sex']],df['agegroup4'],
rownames=['Area','marital status','Disability type','Sex'],
colnames=['Agegroup'],
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 5 MaritalStatus.xlsx')
print('Table 5: MaritalStatus generated and saved')
#############################################################################
xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df['hh_relationship'],
rownames=['Area','Disability type','Sex'],
colnames=['relationship to HH'],
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 6 Head_HH.xlsx')

print('Table 6: Head_HH generated and saved')
#############################################################################
df_total=[]
for col in ['having_disability','hh_size']:

    xtab=pd.crosstab([df['area'],df['hh_type']],df[col],
    rownames=['Area','HH type'],
    colnames=[col],
    values=df['weight'], aggfunc='sum',dropna=False)

    df_total.append(xtab)

T=pd.concat(df_total, axis=1)
T.to_excel('Table 7 HH_Type&Size.xlsx')
print('Table 7: HH_Type&Size generated and saved')
#############################################################################


print('Table 8: Living_Type_Age generated and saved')
#############################################################################


print('Table 9: Living_Type_Age (new) generated and saved')
#############################################################################
cols=[('age5+',"df['dem_ag1']>=5"),('age15+',"df['dem_ag1']>=15"),('age25+',"df['dem_ag1']>=25"),('agegroup6',"df['dem_ag1']>=15")]
df_total=[]

for c in cols:
    #filter the dataframe
    df1=df[eval(c[1])].copy()
    xtab=pd.crosstab([df1['area'],df1['having_disability'],df1['dem_sex']],[df1['ilo_edu_isced11_Yves'],df1[c[0]]],
    rownames=['Area','Disability','Sex'],
    colnames=['edu','age'],
    values=df['weight'], aggfunc='sum',dropna=False)
    df_total.append(xtab)

T=pd.concat(df_total, axis=1)
T.to_excel('Table 10 EducatAttaintment_Age.xlsx')
print('Table 10: EducatAttaintment_Age generated and saved')
#############################################################################


print('Table 11: EducatAttainment_Type generated and saved')
#############################################################################


print('Table 12: School_attendance generated and saved')
#############################################################################


print('Table 13: SchoolAttendance_Type generated and saved')
#############################################################################


print('Table 14: Literacy generated and saved')
#############################################################################


print('Table 15: Literacy_Type_Age generated and saved')
#############################################################################


print('Table 16: ActivityStatus generated and saved')
#############################################################################


print('Table 17: ActivityStatus_Education generated and saved')
#############################################################################


print('Table 18: ActivityStatus_Type generated and saved')
#############################################################################


print('Table 19: LaborForce generated and saved')
#############################################################################


print('Table 20: LaborForce_Type generated and saved')
#############################################################################


print('Table 21: Reason_Not_Seeking_Job generated and saved')
#############################################################################


print('Table 22: Youth_NEET generated and saved')
#############################################################################


print('Table 23: Youth_NEET_Type generated and saved')
#############################################################################


print('Table 24: EducatAttaintment_Age generated and saved')
#############################################################################


print('Table 25: Occupation generated and saved')
#############################################################################


print('Table 26: Sector generated and saved')
#############################################################################


print('Table 27: Sector_type generated and saved')
#############################################################################


print('Table 28: Type_of_ownership generated and saved')
#############################################################################


print('Table 29: Grants_Benefits_Insurance generated and saved')
#############################################################################


Table 1: population_age generated and saved
Table 2: DisabType_Age generated and saved
Table 3: CauseType generated and saved
Table 4: Number_DisabilityDomain generated and saved
Table 5: MaritalStatus generated and saved
Table 6: Head_HH generated and saved
Table 7: HH_Type&Size generated and saved
Table 8: Living_Type_Age generated and saved
Table 9: Living_Type_Age (new) generated and saved
Table 10: EducatAttaintment_Age generated and saved
Table 11: EducatAttainment_Type generated and saved
Table 12: School_attendance generated and saved
Table 13: SchoolAttendance_Type generated and saved
Table 14: Literacy generated and saved
Table 15: Literacy_Type_Age generated and saved
Table 16: ActivityStatus generated and saved
Table 17: ActivityStatus_Education generated and saved
Table 18: ActivityStatus_Type generated and saved
Table 19: LaborForce generated and saved
Table 20: LaborForce_Type generated and saved
Table 21: Reason_Not_Seeking_Job generated and saved
Table 22: Youth_NEET g

In [9]:

col_vals_lfs

{'province': {11.0: 'Duhok',
  12.0: 'Nineveh',
  13.0: 'Sulaymaniyah',
  14.0: 'Kirkuk',
  15.0: 'Erbil',
  16.0: 'Halabja',
  21.0: 'Diyala',
  22.0: 'Anbar',
  23.0: 'Baghdad',
  24.0: 'Babylon',
  25.0: 'Karbala',
  26.0: 'Wasit',
  27.0: 'Salah al-Din',
  28.0: 'Najaf',
  31.0: 'Al-Qadisiyah',
  32.0: 'Muthanna',
  33.0: 'Dhi Qar',
  34.0: 'Maysan',
  35.0: 'Basra'},
 'gharea': {1.0: 'Urban', 2.0: 'Rural'},
 'ghhrpl': {1.0: 'YES', 2.0: 'NO'},
 'dem_more': {1.0: 'YES', 2.0: 'NO'},
 'dem_oth': {1.0: 'YES', 2.0: 'NO'},
 'dem_sex': {1.0: 'MALE', 2.0: 'FEMALE'},
 'dem_rel': {1.0: 'Head',
  2.0: 'Spouse (Wife/Husband)',
  3.0: 'Child (Son/daughter)',
  4.0: 'Grandchild',
  5.0: 'Parent',
  6.0: 'Brother or Sister',
  7.0: 'Son or Daughter in law',
  8.0: 'Other relative',
  9.0: 'Non-relative',
  10.0: 'Domestic worker /Maid/Servant (Live-in)'},
 'dem_bth_year': {9997.0: "DON'T KNOW"},
 'dem_bth_month': {1.0: 'January',
  2.0: 'February',
  3.0: 'March',
  4.0: 'April',
  5.0: 'May',
  

In [7]:
df_total=[]

multiindex_cols=[]

for col in ['MT11','internet_use','MT12']:
    #add labels to values
    if col!='internet_use':
        df[col]=df[col].map(col_vals_wm[col])
        
        r=pd.crosstab([df['HH6'],df['disability_combined']],
        [df[col],df['age_ict']],
        rownames=['Area','Disability level'],colnames=[col_names_wm[col],'age'], 
        values=df['wmweight'],aggfunc='sum',dropna=False)
        df_total.append(r)
        #create the multiindex col and append to multiindex_cols
        '''r.columns are MultiIndex([(         'NO', '15-60'),
                        ('NO RESPONSE', '15-60'),
                        (        'YES', '15-60')],
            names=['Own a mobile phone', 'age'])
            we loop over the tuples and insert at index 0 the column name with col_names_wm[col]'''

        for i in r.columns:
            l=list(i)
            l.insert(0,col_names_wm[col])
            multiindex_cols.append(tuple(l))

In [ ]:
xtab=crosstab()
xtab.generate_xtabs()

In [63]:
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']

df['age_5_1'].value_counts()

5-14     25735
15-24    20936
25-34    13870
nan      11166
35-44    10849
45-54     8653
55-64     4423
65-74     3006
75-84      778
85+        200
Name: age_5_1, dtype: int64

In [60]:
df['dif_sight']

No, no difficulty           87024
Yes, some difficulty         1188
Yes, a lot of difficulty      195
Cannot do it at all            44
Name: dif_sight, dtype: int64

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df=pd.read_excel('C:/Users/511232/Desktop/PM2.5_CC.xlsx')
cols=['Exposure Upper PM 2.5 ug/m3','Exposure Upper PM 2.5 ug/m3_CC']

df.head()

,Country,ISO3,Year,Exposure Upper PM 2.5 ug/m3,Exposure Upper PM 2.5 ug/m3_CC
0,Algeria,DZA,1990,41.6,2
1,Algeria,DZA,1995,44.3,2
2,Algeria,DZA,2000,49.6,2
3,Algeria,DZA,2005,47.5,2
4,Algeria,DZA,2010,44.1,2


In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

df=pd.read_excel('C:/Users/511232/Desktop/PM2.5_CC.xlsx')
cols=['Exposure Upper PM 2.5 ug/m3','Exposure Upper PM 2.5 ug/m3_CC']


def bello(d,col):
    slope=[]
    R2=[]
    country=[]
    indicator=[]

    x=np.array(d['Year']).reshape(-1, 1)
    y=np.array(d[col]).reshape(-1, 1)
    country.append(d['Country'].unique()[0])
    indicator.append(col)

    regr = LinearRegression()
    regr.fit(x, y)

    slope.append(regr.coef_[0][0])
    pred=regr.predict(x)
    R2.append(r2_score(y, pred))
    
    d=pd.DataFrame({'country': country, 'indicator': indicator, 'slope':slope, 'R2':R2})
    df_T.append(d)


df_T=[]
for col in cols:
    df1=df[['Country','ISO3','Year',col]].copy()
    rslt=df.groupby(['Country']).apply(lambda x: bello(x, col))



In [38]:
final=pd.concat(df_T)
final

,country,indicator,slope,R2
0,Algeria,Exposure Upper PM 2.5 ug/m3,0.029284,0.016026
0,Bahrain,Exposure Upper PM 2.5 ug/m3,-0.070646,0.029369
0,Comoros,Exposure Upper PM 2.5 ug/m3,0.027217,0.288594
0,Djibouti,Exposure Upper PM 2.5 ug/m3,0.487121,0.471694
0,Egypt,Exposure Upper PM 2.5 ug/m3,-0.057767,0.008791
0,Iraq,Exposure Upper PM 2.5 ug/m3,-0.037172,0.005966
0,Jordan,Exposure Upper PM 2.5 ug/m3,0.147847,0.220621
0,Kuwait,Exposure Upper PM 2.5 ug/m3,0.150432,0.059870
0,Lebanon,Exposure Upper PM 2.5 ug/m3,0.143228,0.354360
0,Libya,Exposure Upper PM 2.5 ug/m3,0.008801,0.000194
